## **gold-financials-stock-data**

This notebook builds the **`yahoo_stock`** Delta table by joining Yahoo Finance historical stock data with company information from the Silver layer. It creates a unified dataset of stock prices, volumes, and company metadata, and performs an incremental merge to keep the table updated. Robust error handling ensures smooth execution and traceability of issues.

#### **Importing necessary libraries**

In [6]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, 0c94af54-4443-4c81-a759-6395da230bc0, 8, Finished, Available, Finished)

#### **Variables**

In [7]:
file_name = "yfinance_data"
layer_name = "silver"
table_name = "yahoo_stock"

file_path = f"Files/{layer_name}/{file_name}/"

StatementMeta(, 0c94af54-4443-4c81-a759-6395da230bc0, 9, Finished, Available, Finished)

#### **Step 2: Reading the Stock Historical and Information Data**

In [8]:
yf_hist_df = spark.read.format('delta').load(file_path)
com_info_df = spark.read.format('delta').load(f"Files/{layer_name}/comp_info_data/")

StatementMeta(, 0c94af54-4443-4c81-a759-6395da230bc0, 10, Finished, Available, Finished)

#### **Step 3: Select the necessary columns from Stock Historical and Company Information Data**

In [9]:
fy_stock_df = yf_hist_df\
                    .join(broadcast(com_info_df), yf_hist_df.Company == com_info_df.company_name)\
                    .select(
                        col('short_name').alias('company_name'),
                        col('company_image').alias('company_logo'),
                        col('industry'),
                        col('sector'),
                        to_date(col('Date'),'yyyy-MM-dd').alias('stock_date'),
                        col('Open'),
                        col('Close'),
                        col('High'),
                        col('Low'),
                        col('Volume')
                    )

StatementMeta(, 0c94af54-4443-4c81-a759-6395da230bc0, 11, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Gold layer**

In [10]:
try:
    if DeltaTable.isDeltaTable(spark, table_name):
        delta_table = DeltaTable.forPath(spark, table_name)
        
        latest_date = delta_table.toDF().agg({"stock_date": "max"}).collect()[0][0]
        new_data_to_append = fy_stock_df.filter(col("stock_date") > latest_date)
        
        delta_table.alias("target").merge(
            new_data_to_append.alias("source"),
            (col("target.company_name") == col("source.company_name")) &
            (col("target.stock_date") == col("source.stock_date"))
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

    else:
        fy_stock_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, 0c94af54-4443-4c81-a759-6395da230bc0, 12, Finished, Available, Finished)